In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, HuberRegressor
from ipywidgets import interactive, IntSlider, VBox, Label, Layout
from IPython.display import display

# Diagram settings
title_font_size = 12
axis_font_size = 12

# For reproducibility
np.random.seed(42)


def plot_description(text):
    print(f"\nDescription:\n{text}\n")


def polynomial_regression_loss_demo(num_outliers=50, poly_estimator_degree=3):
    # ---------------------------------------------------
    # 1. Create nonlinear dataset (quadratic + noise)
    # ---------------------------------------------------
    np.random.seed(42)
    n_samples = 200
    X = np.linspace(-3, 3, n_samples).reshape(-1, 1)
    y = 3 * X.squeeze()**3 - 3 * X.squeeze()**2 - 4 * X.squeeze() + np.random.randn(n_samples) * 2

    # Add outliers
    outlier_indices = np.random.choice(n_samples, num_outliers, replace=False)
    y[outlier_indices] += 30 * np.random.randn(num_outliers)
    
    # Polynomial features
    poly = PolynomialFeatures(degree=poly_estimator_degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    
    # Normalize for fairness
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_poly)
    
    # ---------------------------------------------------
    # 2. Train models with different loss functions
    # ---------------------------------------------------
    ols = LinearRegression().fit(X_scaled, y)
    sgd_mae = SGDRegressor(loss='epsilon_insensitive', epsilon=0.0, 
                           max_iter=10000, tol=1e-3, random_state=42).fit(X_scaled, y)
    huber_reg = HuberRegressor(epsilon=1.0, max_iter=1000).fit(X_scaled, y)
    
    # ---------------------------------------------------
    # 3. Predict on dense grid for smooth curves
    # ---------------------------------------------------
    X_line = np.linspace(X.min(), X.max(), 200).reshape(-1, 1)
    X_line_poly = poly.transform(X_line)
    X_line_scaled = scaler.transform(X_line_poly)
    
    y_ols = ols.predict(X_line_scaled)
    y_mae = sgd_mae.predict(X_line_scaled)
    y_huber = huber_reg.predict(X_line_scaled)
    
    # ---------------------------------------------------
    # 4. Visualization
    # ---------------------------------------------------
    plt.figure(figsize=(8, 6))
    plt.scatter(X, y, color='gray', alpha=0.4, label="Data")
    plt.plot(X_line, y_ols, label="MSE", linewidth=2)
    plt.plot(X_line, y_mae, label="MAE", linewidth=2)
    plt.plot(X_line, y_huber, label="Huber", linewidth=2)
    
    plt.title("Polynomial Regression with Different Loss Functions", fontsize=16)
    plt.xlabel("X", fontsize=14)
    plt.ylabel("y", fontsize=14)
    plt.ylim([-125,100])
    plt.legend()
    plt.grid(True)
    plt.show()


# ---------------------------------------------------
# 5. Interactive control panel (same style as before)
# ---------------------------------------------------
def polynomial_regression_loss_demo_interact():
    plot_description("Three polynomial models (with MSE/MAE/Huber loss) are fitted to the data. Play around with the"
                     " number of outliers and the polynomial degree of the models. Notice differences in the models."
                    " You will see how the Huber loss model keeps balance between the MSE and MAE loss model.")
    
    num_outliers_slider = IntSlider(
        value=10, min=0, max=100, step=5,
        description="Number of Outliers",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    poly_estimator_degree = IntSlider(
        value=3, min=1, max=6, step=1,
        description="Polyn. Estimator Degree",
        style={'description_width': '150px'},
        layout=Layout(width='500px')
    )

    ui_box = VBox([
        Label(value="📊 Controls", layout=Layout(margin="0 0 0 0")),
    ])

    interactive_plot = interactive(
        polynomial_regression_loss_demo,
        num_outliers=num_outliers_slider,
        poly_estimator_degree=poly_estimator_degree
    )

    display(ui_box, interactive_plot)